In [ ]:
from bids import BIDSLayout

In [2]:
root = '/data2/project_modelbasedMVPA/bornstein2017'

In [3]:
layout = BIDSLayout(root,derivatives=True)

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/bids/layout/models.py:152: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


In [7]:
layout.derivatives['fMRIPrep'].get(suffix='bold',
                                        space='MNI152NLin2009cAsym',
                                        task='learn',
                                        extension="nii.gz",
                                  session=None)

[<BIDSImageFile filename='/data2/project_modelbasedMVPA/bornstein2017/derivatives/fmriprep/sub-01/func/sub-01_task-learn_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'>,
 <BIDSImageFile filename='/data2/project_modelbasedMVPA/bornstein2017/derivatives/fmriprep/sub-02/func/sub-02_task-learn_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'>,
 <BIDSImageFile filename='/data2/project_modelbasedMVPA/bornstein2017/derivatives/fmriprep/sub-03/func/sub-03_task-learn_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'>,
 <BIDSImageFile filename='/data2/project_modelbasedMVPA/bornstein2017/derivatives/fmriprep/sub-04/func/sub-04_task-learn_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'>,
 <BIDSImageFile filename='/data2/project_modelbasedMVPA/bornstein2017/derivatives/fmriprep/sub-05/func/sub-05_task-learn_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'>,
 <BIDSImageFile filename='/data2/project_modelbasedMVPA/bornstein2017/derivatives/fmriprep/sub-06/func/sub-06_task-learn_spac

In [2]:
import pandas as pd

In [3]:
from mbmvpa.preprocessing.computational_modeling import banditNarm_4par as modelling_module

In [4]:
from mbmvpa.utils.bids_utils import BIDSConEventsOnly

In [5]:
from mbmvpa.preprocessing.events import LatentProcessGenerator

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/nilearn/glm/__init__.py:56: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  'It may change in any future release of Nilearn.', FutureWarning)


In [35]:
!ls /data2/project_modelbasedMVPA/

bornstein2017	   fmriprep.sh		    PRL		    temp_2
ds000005	   piva_dd		    sample	    tom
ds001499-download  piva_dd_mbmvpa	    schonberg_bart
fmriprep-all	   piva_dd_mbmvpa_zoomed_2  temp


In [7]:
root = '/data2/project_modelbasedMVPA/ds001607-download'

In [8]:
task_name = 'learn'

In [9]:
layout = BIDSLayout(root)

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/bids/layout/models.py:152: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


In [10]:
ex_events_file= layout.get(task='learn',suffix='events')[0]

In [11]:
ex_events_file

<BIDSDataFile filename='/data2/project_modelbasedMVPA/ds001607-download/sub-01/func/sub-01_task-learn_events.tsv'>

In [12]:
df_events = pd.read_table(ex_events_file)

In [13]:
df_events.head()

,onset,duration,type,choice,rwdval,RT
0,2063751.003,3,bandit,1,0,1.2439
1,2063757.986,3,bandit,2,10,0.7019
2,2063764.969,3,bandit,3,0,0.9651
3,2063771.952,3,bandit,3,10,1.0103
4,2063778.936,3,bandit,2,0,0.4705


In [14]:
df_events[df_events['rwdval'] > 0]['rwdval'] = 1

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
def adjust_function_dfwise(df_events):
    df_events['onset'] = df_events['onset']-df_events['onset'].min() #+df_events['RT']+2
    df_events['time_feedback'] = df_events['onset'] + df_events['RT']+2
    gain = []
    loss = []
    for _, row in df_events.iterrows():
        if row['rwdval'] > 0:
            gain.append(1)
            loss.append(0)
        else:
            gain.append(0)
            loss.append(-1)
    df_events['gain'] = gain
    df_events['loss'] = loss
    return df_events

In [27]:
def filter_function(row):
    return row['choice']in [1,2,3]

In [16]:
process_name = 'QRchosen'

In [17]:
latent_function_dfwise = modelling_module.ComputationalModel(process_name)

In [18]:
bids_controller = BIDSConEventsOnly(bids_layout=root,
                                            subjects='all',
                                            task_name=task_name)

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/bids/layout/models.py:152: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


INFO: start loading BIDSLayout
INFO: MB-MVPA is loaded
INFO: target subjects-all
INFO: 32 file(s) in Original


In [28]:
y_generator = LatentProcessGenerator(bids_layout=root,
                                     bids_controller=bids_controller,
                                    adjust_function_dfwise=adjust_function_dfwise,
                                     filter_function=filter_function,
                                    dm_model = 'banditNarm_4par',
                                    task_name = task_name,
                                    process_name = process_name,
                                    use_1sec_duration=True,
                                    ignore_fmriprep=True)

In [29]:
df_events_list, event_infos_list = y_generator._init_df_events_from_bids()

In [30]:
df_events_list = [y_generator.filter_function_dfwise(
                                        y_generator.adjust_function_dfwise(df_events)) for df_events in df_events_list]

In [31]:
df_events_list[0]['choice'].unique()

array([1, 2, 3], dtype=object)

In [32]:
df_events = pd.concat(df_events_list)

In [33]:
df_events['choice'].unique()

array([1, 2, 3], dtype=object)

In [34]:
y_generator.set_computational_model()

INFO: running computational model [hBayesDM-banditNarm_4par]
Using cached StanModel: cached-banditNarm_4par-pystan_2.19.1.1.pkl



Model  = banditNarm_4par
Data   = <pandas.DataFrame object>

Details:
 # of chains                    = 4
 # of cores used                = 4
 # of MCMC samples (per chain)  = 4000
 # of burn-in samples           = 1000
 # of subjects                  = 32
 # of (max) trials per subject  = 180

Using cached StanModel: cached-banditNarm_4par-pystan_2.19.1.1.pkl
Index(['onset', 'duration', 'type', 'choice', 'rwdval', 'rt', 'trial',
       'subjid', 'run', 'session', 'timefeedback', 'gain', 'loss'],
      dtype='object')
['onset', 'duration', 'type', 'choice', 'rwdval', 'RT', 'trial', 'subjID', 'run', 'session', 'time_feedback', 'gain', 'loss']
************************************
**** Model fitting is complete! ****
************************************
